In [1]:
import autogen
from autogen import ConversableAgent

In [2]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-5-mini","api_key":OPENAI_API_KEY}

In [3]:
from autogen import UserProxyAgent

In [4]:
import pprint

In [5]:
key_clause_extractor = ConversableAgent( name = "key_clause_extractor_Agent",
                                         system_message=""" You are an key clause extractor agent, your role is to review the legal content submitted and 
                                         extract the key clauses from the content""",
                                         llm_config=llm_config,
                                         human_input_mode="NEVER")

Risk_Flagger= ConversableAgent( name= "Risk_flagger_Agent",
                                system_message=""" You are an Risk Flagger Agent, your role is to flag any risk or harmful contents from the legal
                                contents""",
                                llm_config=llm_config,
                                human_input_mode="NEVER")

Final_reviewer= ConversableAgent( name= "Final_reviewer_Agent",
                                system_message=""" Final_reviewer_Agent, your role is to verify the entire legal content, and suggest any improvements if required 
                                and highlight any risky contents""",
                                llm_config=llm_config,
                                human_input_mode="NEVER")

In [6]:
User_proxy= UserProxyAgent( name ="UserProxyAgent",
                            human_input_mode="ALWAYS",
                            code_execution_config=False)
                              

In [7]:
cordinator= ConversableAgent( name = "cordinator_Agent",
                                 system_message="""Your role is like a mediator who gets the legal content from user proxy and pass it on to individual agents
                                 which will process the content further""",
                                 llm_config=llm_config,
                                 human_input_mode="NEVER")

message = """The Receiving Party agrees to treat all non-public information received
    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.
    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.
    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.
    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials."""

"sender" : User_proxy,
               "recipient" : cordinator,
               "message" : """The Receiving Party agrees to treat all non-public information received
    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.
    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.
    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.
    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.""",
               "max_turns": 1,
               "summary_method": "last_msg",

               { "sender" : cordinator,
               "recipient" : User_proxy,
               "max_turns": 1,
               #"carryover": [0,1,2,3],
               "summary_method": "last_msg"
             },

message= cordinator.chat_messages_for_summary(User_proxy)[-1]['content']
message

chat_flow= [ 
             { "sender" : cordinator,
               "recipient" : key_clause_extractor,
               "max_turns": 1,
               "message" : message,
               "summary_method": "reflection_with_llm",
             },
             { "sender" : cordinator,
               "recipient" : Risk_Flagger,
               "message" : message,
               "max_turns": 1,
               "summary_method": "reflection_with_llm",
             },
             { "sender" : cordinator,
               "recipient" : Final_reviewer,
               "message" : message,
               "carryover": [0,1],
               "max_turns": 1,
               "summary_method": "reflection_with_llm",
             },
             
           ]

IndexError                                Traceback (most recent call last)
Cell In[8], line 1
----> 1 message= cordinator.chat_messages_for_summary(User_proxy)[-1]['content']
      2 message
      4 chat_flow= [ 
      5              { "sender" : cordinator,
      6                "recipient" : key_clause_extractor,
   (...)     24 
     25            ]

IndexError: list index out of range

# ERROR TO BE NOTED

File C:\anaconda3\envs\agenticai\Lib\site-packages\autogen\agentchat\conversable_agent.py:830, in ConversableAgent._summary_from_nested_chats(chat_queue, recipient, messages, sender, config)
    825 # Process carryover configuration
    826 restore_chat_queue_message, original_chat_queue_message = ConversableAgent._process_chat_queue_carryover(
    827     chat_queue, recipient, messages, sender, config
    828 )
--> 830 chat_to_run = ConversableAgent._get_chats_to_run(chat_queue, recipient, messages, sender, config)
    831 if not chat_to_run:
    832     return True, None

File C:\anaconda3\envs\agenticai\Lib\site-packages\autogen\agentchat\conversable_agent.py:652, in ConversableAgent._get_chats_to_run(chat_queue, recipient, messages, sender, config)
    650     message = last_msg
    651 if callable(message):
--> 652     message = message(recipient, messages, sender, config)
    653 # We only run chat that has a valid message. NOTE: This is prone to change depending on applications.
    654 if message:

TypeError: message() takes 2 positional arguments but 4 were given

The Source of Each Argument
When the internal method _get_chats_to_run encounters your callable, it pulls the data from the current state of the manager agent:

recipient: This is the agent object defined in your chat_queue or nested_chat configuration for that specific step.

messages: This is the most important one. It is pulled from the chat_res.chat_history of the previous chat in the sequence. If it's the very first chat, this might be an empty list or contain the initial sender's message.

sender: This is the agent that is initiating the nested chat (usually self from the perspective of the ConversableAgent running the queue).

config: This is passed from the carryover or the chat_configuration dictionary you provided when you first called initiate_chats.

In [8]:
# the messages argument in message function holds the conversation summary between the cordinator an user proxy 
# and it does not append the next conversations automatically unless its done manually
# conversation thread between each agent is seperate

def message(recipient,messages,sender,config):
    message= sender.chat_messages_for_summary(recipient)[-1]['content']
    return message

chat_flow= [ 
             { "sender" : cordinator,
               "recipient" : key_clause_extractor,
               "max_turns": 1,
               "message" : message,
               "summary_method": "reflection_with_llm",
             },
             { "sender" : cordinator,
               "recipient" : Risk_Flagger,
               "message" : message,
               "max_turns": 1,
               "summary_method": "reflection_with_llm",
             },
             { "sender" : cordinator,
               "recipient" : Final_reviewer,
               "message" : message,
               "carryover": [0,1],
               "max_turns": 1,
               "summary_method": "reflection_with_llm",
             },
             
           ]

In [9]:
def message(recipient, messages, sender, config):
    """
    Robustly retrieves the input document from the current chat history.
    Takes 4 arguments as required by AutoGen's internal caller.
    """
    if not messages:
        return "No input provided. Please provide the document to review."
    
    # We take the last message from the coordinator's history with the user
    # to ensure the sub-agents are working on the most recent input.
    last_msg = messages[-1].get("content", "")
    return last_msg

In [10]:
def check_and_transfer(recipient, messages, sender, config):
    """
    Acts as the 'Failure Detection' logic. 
    Checks if the previous agent actually succeeded.
    """
    if not messages:
        return "System Error: No history found."

    last_reply = messages[-1].get("content", "")

    # FAILURE DETECTION LOGIC
    # If the previous agent failed to find anything or errored out
    if len(last_reply) < 10 or "error" in last_reply.lower():
        # ESCALATION/FALLBACK PATH
        return "FAILURE_DETECTED: Escalating to Human Reviewer. The automated risk detection failed to process this document."
    
    # SUCCESS PATH: Pass the findings to the next agent
    return last_reply

# The Chain
chat_queue = [
    {
        "sender": cordinator,
        "recipient": key_clause_extractor,
        "message": message, # Uses the document loader function from previous step
        "summary_method": "last_msg",
        "max_turns": 1,
    },
    {
        "sender": cordinator,
        "recipient": Risk_Flagger,
        "message": check_and_transfer, # Inspects Extractor output
        "summary_method": "last_msg",
        "max_turns": 1,
    },
    {
        "sender": cordinator,
        "recipient": Final_reviewer,
        "message": check_and_transfer, # Inspects Risk output
        "carryover": [0, 1], # Feeds summaries to the human-like final review
        "max_turns": 1,
    }
]

In [11]:
cordinator.register_nested_chats(chat_queue, trigger= User_proxy)

In [12]:
#from autogen import initiate_chat

chats= cordinator.initiate_chat(User_proxy, message=" Please provide the legal content", max_turns=2)



cordinator_Agent (to UserProxyAgent):

 Please provide the legal content

--------------------------------------------------------------------------------


Replying as UserProxyAgent. Provide feedback to cordinator_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  "max_turns": 1The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.


UserProxyAgent (to cordinator_Agent):

"max_turns": 1The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
cordinator_Agent (to key_clause_extractor_Agent):

"max_turns": 1The Receiving Pa

Replying as UserProxyAgent. Provide feedback to cordinator_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  0k


UserProxyAgent (to cordinator_Agent):

0k

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (7a725886-5da5-4a44-b980-a1351ab7d11c): Maximum turns (2) reached


In [13]:
cordinator.chat_messages

defaultdict(list,
            {<autogen.agentchat.user_proxy_agent.UserProxyAgent at 0x209758a1510>: [{'content': ' Please provide the legal content',
               'role': 'assistant',
               'name': 'cordinator_Agent'},
              {'content': '"max_turns": 1The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.',
               'role': 'user',
               'name': 'UserProxyAgent'},
              {'content': 'I’ve reviewed the NDA excerpt 

In [14]:
cordinator.chat_messages_for_summary(User_proxy)

[{'content': ' Please provide the legal content',
  'role': 'assistant',
  'name': 'cordinator_Agent'},
 {'content': '"max_turns": 1The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.',
  'role': 'user',
  'name': 'UserProxyAgent'},
 {'content': 'I’ve reviewed the NDA excerpt and the prior agent’s analysis. Below I confirm and expand on the key risks, ambiguities and missing protections, and provide concrete suggested wording you can use to fix them. I

In [15]:
cordinator.chat_messages_for_summary(User_proxy)[-1]['content']

'0k'

In [17]:
dir(chat_queue)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [ ]:
chats.chat_history